In [1]:
import pandas as pd
import re 
import seaborn as sns
import numpy as np

In [2]:
tw_clean = pd.read_csv("tw_data_short.csv")
tw_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 567526 entries, 0 to 567525
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   date            567526 non-null  object 
 1   time            567526 non-null  object 
 2   date_final      567526 non-null  object 
 3   tweet           567526 non-null  object 
 4   mentions        567526 non-null  object 
 5   replies_count   567526 non-null  float64
 6   retweets_count  567526 non-null  float64
 7   likes_count     567526 non-null  float64
 8   hashtags        567526 non-null  object 
dtypes: float64(3), object(6)
memory usage: 39.0+ MB


In [3]:
tw_tokens = pd.read_csv("tweets_token.csv")
tw_tokens.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 567526 entries, 0 to 567525
Data columns (total 100 columns):
 #   Column             Non-Null Count   Dtype
---  ------             --------------   -----
 0   Unnamed: 0         567526 non-null  int64
 1   dogecoin           567526 non-null  int64
 2   dogearmy           567526 non-null  int64
 3   elonmusk           567526 non-null  int64
 4   buy                567526 non-null  int64
 5   dogecointothemoon  567526 non-null  int64
 6   btc                567526 non-null  int64
 7   crypto             567526 non-null  int64
 8   dogecoinrise       567526 non-null  int64
 9   dogecoinarmy       567526 non-null  int64
 10  eth                567526 non-null  int64
 11  get                567526 non-null  int64
 12  bitcoin            567526 non-null  int64
 13  moon               567526 non-null  int64
 14  dollar             567526 non-null  int64
 15  coin               567526 non-null  int64
 16  one                567526 non-null  i

In [4]:
all_data_token = tw_clean.join(tw_tokens, how='left', lsuffix='l', rsuffix='r', sort=False )
all_data_token["date"] = pd.to_datetime(all_data_token["date"])
all_data_token["timel"] = pd.to_datetime(all_data_token["timel"], format= '%H:%M:%S' ).dt.time
all_data_token["date_final"] = pd.to_datetime(all_data_token["date_final"])
all_data_token.rename(columns={"date_final": "time"}, inplace=True)
all_data_token

,date,timel,time,tweetl,mentions,replies_count,retweets_count,likes_count,hashtags,Unnamed: 0,...,coinbase,take,cent,tesla,big,got,dogeto,dip,wallet,way
0,2021-04-28,16:59:39,2021-04-29 00:59:39,Checking doge coin seeing if ur rich yet 👀,[],1.0,1.0,8.0,[],0,...,0,0,0,0,0,0,0,0,0,0
1,2021-04-28,16:59:31,2021-04-29 00:59:31,Your so doge,[],2.0,3.0,7.0,[],1,...,0,0,0,0,0,0,0,0,0,0
2,2021-04-28,16:59:24,2021-04-29 00:59:24,#DOGE Move Right Now 🐕💹🚀 Follow The best Chan...,[],31.0,18.0,175.0,"['doge', 'telegram']",2,...,0,0,0,0,0,0,0,0,0,0
3,2021-04-28,16:59:12,2021-04-29 00:59:12,@Seeker_Good 1000x for all dogseason coins $s...,[],0.0,1.0,1.0,[],3,...,0,0,0,0,0,0,0,0,0,0
4,2021-04-28,16:59:04,2021-04-29 00:59:04,"Oh and by the way, we've just burned over 2897...",[],74.0,179.0,342.0,"['memecoin', 'solana', 'solsummer', 'solszn', ...",4,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
567521,2021-01-01,16:15:57,2021-01-02 00:15:57,#doge EVERYONE DIMP $20 to #dogecoin tonight!!...,"[{'screen_name': 'elonmusk', 'name': 'elon mus...",0.0,4.0,10.0,"['doge', 'dogecoin']",567521,...,0,0,0,0,0,0,0,0,0,0
567522,2021-01-01,16:15:38,2021-01-02 00:15:38,1 DOGE Price: 0.00000016 BTC #doge #dogecoin 2...,[],0.0,1.0,1.0,"['doge', 'dogecoin']",567522,...,0,0,0,0,0,0,0,0,0,0
567523,2021-01-01,16:09:03,2021-01-02 00:09:03,"@Flak And to celebrate, have some Doge Caek! ...",[],1.0,1.0,12.0,[],567523,...,0,0,0,0,0,0,0,0,0,0
567524,2021-01-01,16:00:29,2021-01-02 00:00:29,@dogecoin Pump it doge.not only in mere words....,[],2.0,2.0,22.0,[],567524,...,0,0,0,0,0,0,0,0,0,0


In [5]:
all_data_token.drop(labels = ["hashtags", "Unnamed: 0"], axis=1, inplace=True)

In [6]:
#Adding col with 0 1 values, wich mean more or less 100 likes
all_data_token.insert(7,'top_tw', '')
all_data_token['top_tw'] = pd.DataFrame(np.where(all_data_token['likes_count']>100, '1', '0'))
all_data_token['top_tw'].value_counts()

0    545461
1     22065
Name: top_tw, dtype: int64

In [7]:
all_data_token["top_tw"] = all_data_token["top_tw"].apply(pd.to_numeric)

In [8]:
all_data_token.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 567526 entries, 0 to 567525
Columns: 108 entries, date to way
dtypes: datetime64[ns](2), float64(3), int64(100), object(3)
memory usage: 467.6+ MB


In [9]:
dict_cols = {i : "sum" for i in all_data_token.columns[5:-1]}
dict_cols

{'replies_count': 'sum',
 'retweets_count': 'sum',
 'top_tw': 'sum',
 'likes_count': 'sum',
 'dogecoin': 'sum',
 'dogearmy': 'sum',
 'elonmusk': 'sum',
 'buy': 'sum',
 'dogecointothemoon': 'sum',
 'btc': 'sum',
 'crypto': 'sum',
 'dogecoinrise': 'sum',
 'dogecoinarmy': 'sum',
 'eth': 'sum',
 'get': 'sum',
 'bitcoin': 'sum',
 'moon': 'sum',
 'dollar': 'sum',
 'coin': 'sum',
 'one': 'sum',
 'like': 'sum',
 'xrp': 'sum',
 'let': 'sum',
 'go': 'sum',
 'hold': 'sum',
 'k': 'sum',
 'people': 'sum',
 'dogecoins': 'sum',
 'going': 'sum',
 'cryptocurrency': 'sum',
 'dogecointo': 'sum',
 'dogetothemoon': 'sum',
 'price': 'sum',
 'dogeday': 'sum',
 'ada': 'sum',
 'today': 'sum',
 'dogecointoadollar': 'sum',
 'timer': 'sum',
 'bnb': 'sum',
 'link': 'sum',
 'amp': 'sum',
 'hodl': 'sum',
 'make': 'sum',
 'last': 'sum',
 'day': 'sum',
 'pump': 'sum',
 'sell': 'sum',
 'money': 'sum',
 'see': 'sum',
 'elon': 'sum',
 'new': 'sum',
 'retweet': 'sum',
 'ltc': 'sum',
 'amc': 'sum',
 'good': 'sum',
 'u': 's

In [10]:
#grouping by date and hour
data_agg = all_data_token.groupby([all_data_token['time'].dt.date, all_data_token['time'].dt.hour]).agg(dict_cols)

In [11]:
data_agg

replies_count  retweets_count  top_tw  likes_count  dogecoin  \
time       time                                                                 
2021-01-01 8               0.0             8.0       0         16.0         4   
           9              36.0           124.0       0        107.0         0   
           10             52.0            80.0       0         69.0         4   
           11             19.0            46.0       0         43.0         0   
           12             12.0            30.0       0         64.0         3   
...                        ...             ...     ...          ...       ...   
2021-04-29 3             207.0           963.0       7       4413.0        65   
           4             643.0          1311.0       5       3182.0        63   
           5             180.0           861.0       3       2090.0        71   
           6            1360.0          3004.0       5      13427.0        54   
           7            1886.0          5958.0      40      31404.0       173   

                 dogearmy  elonmusk  buy  dogecointothemoon  btc  ...  hit  \
time       time                                                   ...        
2021-01-01 8            0         1    0                  0    2  ...    0   
           9            0         0    0                  0    0  ...    0   
           10           0         0    0                  0    2  ...    0   
           11           0         0    0                  0    1  ...    0   
           12           0         1    2                  0    3  ...    0   
...                   ...       ...  ...                ...  ...  ...  ...   
2021-04-29 3           31        12    9                 14   16  ...    3   
           4           24        16   13                 12   27  ...    3   
           5           25        25    4                 12   19  ...    5   
           6           30        14    6                  7   26  ...    0   
           7           77       185   31                 34   72  ...   10   

                 coinbase  take  cent  tesla  big  got  dogeto  dip  wallet  
time       time                                                              
2021-01-01 8            0     0     0      0    0    0       0    0       0  
           9            0     0     0      0    2    0       0    0       3  
           10           0     0     0      0    0    2       0    0       3  
           11           0     0     0      0    0    0       0    0       3  
           12           0     0     0      0    0    0       0    0       2  
...                   ...   ...   ...    ...  ...  ...     ...  ...     ...  
2021-04-29 3            2     3     1      3    2    7       0    0       7  
           4            5     3     2      5    7    4       0    2       7  
           5            1     9     0      3    1    8       0    3       7  
           6            3     2     3      4    1    3       0    0       5  
           7            1     9     7      8   13    5       0    5       7  

[2830 rows x 102 columns]

In [12]:
data_prices = pd.read_csv("data/dataprices.csv")

In [13]:
data_prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16875 entries, 0 to 16874
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   time    16875 non-null  object 
 1   open    16875 non-null  float64
 2   high    16875 non-null  float64
 3   low     16875 non-null  float64
 4   Volume  16875 non-null  float64
 5   tv      16875 non-null  int64  
dtypes: float64(4), int64(1), object(1)
memory usage: 791.1+ KB


In [14]:
data_prices['time'] = pd.to_datetime(data_prices['time'])
data_prices.head()

,time,open,high,low,Volume,tv
0,2021-01-01 00:00:00,0.004683,0.004702,0.004616,1865676.0,1
1,2021-01-01 01:00:00,0.004705,0.004766,0.004701,1082226.0,1
2,2021-01-01 02:00:00,0.004733,0.004765,0.004733,85161.0,2
3,2021-01-01 03:00:00,0.004754,0.005001,0.004754,5030766.0,5
4,2021-01-01 04:00:00,0.005000,0.005001,0.004816,5531754.0,-4


In [15]:
data_prices = data_prices.loc[(data_prices["time"] > "2021-01-02 00:00:00") & (data_prices["time"] < "2021-04-29 01:00:00")]

In [16]:
data_prices

,time,open,high,low,Volume,tv
25,2021-01-02 01:00:00,0.005818,0.005854,0.005564,11123815.0,-3
26,2021-01-02 02:00:00,0.005754,0.006431,0.005701,32955177.0,5
27,2021-01-02 03:00:00,0.006353,0.006791,0.006066,108807546.0,5
28,2021-01-02 04:00:00,0.006670,0.007981,0.006654,166272032.0,5
29,2021-01-02 05:00:00,0.007756,0.007776,0.006965,90792179.0,-5
...,...,...,...,...,...,...
2820,2021-04-28 20:00:00,0.310310,0.310760,0.301330,19680380.6,-3
2821,2021-04-28 21:00:00,0.306520,0.315320,0.304800,20889588.2,3
2822,2021-04-28 22:00:00,0.310590,0.326530,0.308500,32387298.2,5
2823,2021-04-28 23:00:00,0.321110,0.326890,0.316570,33777982.1,2


In [18]:
data_prices = data_prices.groupby([data_prices['time'].dt.date, data_prices['time'].dt.hour]).agg({"open": "sum", "high": "sum", "low": "sum", "tv": "sum", "Volume": "sum"})


In [20]:
pd.DataFrame(data_prices)

open      high       low  tv       Volume
time       time                                               
2021-01-02 1     0.005818  0.005854  0.005564  -3   11123815.0
           2     0.005754  0.006431  0.005701   5   32955177.0
           3     0.006353  0.006791  0.006066   5  108807546.0
           4     0.006670  0.007981  0.006654   5  166272032.0
           5     0.007756  0.007776  0.006965  -5   90792179.0
...                   ...       ...       ...  ..          ...
2021-04-28 20    0.310310  0.310760  0.301330  -3   19680380.6
           21    0.306520  0.315320  0.304800   3   20889588.2
           22    0.310590  0.326530  0.308500   5   32387298.2
           23    0.321110  0.326890  0.316570   2   33777982.1
2021-04-29 0     0.323320  0.323320  0.306930  -5   34687125.8

[2800 rows x 5 columns]

In [27]:
data_prices.describe()

,open,high,low,tv,Volume
count,2800.000000,2800.000000,2800.000000,2800.000000,2.800000e+03
mean,0.072687,0.074621,0.070898,0.024286,2.668762e+07
std,0.083215,0.086436,0.080163,3.120559,5.713087e+07
min,0.005754,0.005854,0.005564,-5.000000,9.028900e+04
25%,0.032592,0.033352,0.031404,-2.000000,3.230013e+06
50%,0.054496,0.055097,0.054081,0.000000,7.743748e+06
75%,0.060882,0.061654,0.059733,2.000000,2.486749e+07
max,0.425267,0.480000,0.404310,5.000000,9.334161e+08


In [22]:
#merging data with pricess
all_data_m = data_agg.merge(data_prices, how = "inner", left_index = True, right_index = True)

In [29]:
all_data_m.to_csv("data/all_data_m.csv")

In [28]:
all_data_m.head()

replies_count  retweets_count  top_tw  likes_count  dogecoin  \
time       time                                                                 
2021-01-02 1              17.0            34.0       0        149.0         5   
           2               8.0            90.0       0         70.0         5   
           3               9.0            43.0       0        133.0         8   
           4              32.0           109.0       0        277.0        11   
           5              28.0            90.0       1        382.0         3   

                 dogearmy  elonmusk  buy  dogecointothemoon  btc  ...  big  \
time       time                                                   ...        
2021-01-02 1            0         1    1                  0    2  ...    0   
           2            0         1    1                  0    0  ...    0   
           3            0         0    0                  0    1  ...    2   
           4            0         4    0                  0    1  ...    0   
           5            0         0    2                  0    9  ...    0   

                 got  dogeto  dip  wallet      open      high       low  tv  \
time       time                                                               
2021-01-02 1       1       0    0       0  0.005818  0.005854  0.005564  -3   
           2       0       0    0       3  0.005754  0.006431  0.005701   5   
           3       0       0    0       0  0.006353  0.006791  0.006066   5   
           4       0       0    0       4  0.006670  0.007981  0.006654   5   
           5       0       0    0       0  0.007756  0.007776  0.006965  -5   

                      Volume  
time       time               
2021-01-02 1      11123815.0  
           2      32955177.0  
           3     108807546.0  
           4     166272032.0  
           5      90792179.0  

[5 rows x 107 columns]